In [8]:
!pip install pandas numpy torch transformers datasets scikit-learn
!pip install accelerate

In [10]:
import pandas as pd
import json
import torch
import torch.nn as nn
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ------------------------------
# ✅ Load & Preprocess Data
# ------------------------------

def load_full_and_filter_data(file_path):
    print(f"Loading full dataset from {file_path}...")
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_data = json.loads(line.strip())
                data.append(json_data)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON: {line[:100]}")
    
    df = pd.DataFrame(data)
    print(f"✅ Loaded full dataset: {len(df)} rows")

    # Apply filtering AFTER loading
    df = df[(df["verifiable"] == "VERIFIABLE") & (df["label"].isin(["SUPPORTS", "REFUTES"]))]
    print(f"✅ Filtered dataset: {len(df)} rows (Only VERIFIABLE + SUPPORTS/REFUTES)")

    return df

# Paths
train_file_path = "train.jsonl"
dev_file_path = "shared_task_dev.jsonl"

# Load and filter datasets **without predefined sample size**
print("\nStarting dataset loading process...")
train_df = load_full_and_filter_data(train_file_path)
dev_df = load_full_and_filter_data(dev_file_path)
print("✅ Datasets fully loaded and filtered!")

# ------------------------------
# ✅ Label Conversion & Class Weighting
# ------------------------------

print("\nMapping labels to binary values...")
label_map = {"SUPPORTS": 1, "REFUTES": 0}
train_df['label'] = train_df['label'].map(label_map)
dev_df['label'] = dev_df['label'].map(label_map)
print("✅ Label mapping complete!")

# ✅ FIX: Compute class weights correctly using np.array
class_weights = compute_class_weight("balanced", classes=np.array([0, 1]), y=train_df['label'].values)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# ------------------------------
# ✅ Tokenization & Dataset Conversion
# ------------------------------

print("\nExtracting texts and labels for tokenization...")
train_texts, train_labels = train_df['claim'].tolist(), train_df['label'].tolist()
dev_texts, dev_labels = dev_df['claim'].tolist(), dev_df['label'].tolist()
print("✅ Text extraction complete!")

print("\nInitializing BERT tokenizer...")
model_name = "bert-base-uncased"  # ✅ Using BERT instead of RoBERTa
tokenizer = BertTokenizer.from_pretrained(model_name)

print("Tokenizing datasets...")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
print("✅ Tokenization complete!")

train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"].tolist(),
    "attention_mask": train_encodings["attention_mask"].tolist(),
    "labels": train_labels
})
dev_dataset = Dataset.from_dict({
    "input_ids": dev_encodings["input_ids"].tolist(),
    "attention_mask": dev_encodings["attention_mask"].tolist(),
    "labels": dev_labels
})

# ------------------------------
# ✅ Load BERT Model (With Weighted Loss)
# ------------------------------

print("\nLoading BERT model with weighted loss...")
class WeightedBERT(nn.Module):
    def __init__(self, model_name, class_weights):
        super().__init__()
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}

bert_model = WeightedBERT(model_name, class_weights)
print("\n✅ BERT Model with Weighted Loss Loaded!")

# ------------------------------
# ✅ Training Arguments & Trainer
# ------------------------------

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True,
    learning_rate=2e-5,  # Optimized learning rate
    lr_scheduler_type="linear",
    fp16=True if torch.cuda.is_available() else False,  # 🔥 Mixed precision for faster training
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
print("\n✅ BERT model training complete!")



Starting dataset loading process...
Loading full dataset from train.jsonl...
✅ Loaded full dataset: 145449 rows
✅ Filtered dataset: 109810 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
Loading full dataset from shared_task_dev.jsonl...
✅ Loaded full dataset: 19998 rows
✅ Filtered dataset: 13332 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
✅ Datasets fully loaded and filtered!

Mapping labels to binary values...
✅ Label mapping complete!

Extracting texts and labels for tokenization...
✅ Text extraction complete!

Initializing BERT tokenizer...
Tokenizing datasets...
✅ Tokenization complete!

Loading BERT model with weighted loss...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/common/home/projectgrps/IS424/IS424G4/jupyterlab-venv-pytorch-240/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



✅ BERT Model with Weighted Loss Loaded!


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.397600,0.463878,0.799655,0.805164,0.783615,0.827933
2,0.276000,0.542983,0.799730,0.809612,0.771541,0.851635
3,0.289600,0.699716,0.802355,0.815437,0.764814,0.873237
4,0.292200,0.787990,0.797930,0.808855,0.767367,0.855086
5,0.177500,1.083317,0.793579,0.812278,0.744809,0.893189



✅ BERT model training complete!


In [ ]:
🔍 Interpretation of the Results
1️⃣ Accuracy & F1-Score Remain Stable (~80%)
The accuracy hovers around 79.3% - 80.2%, which is solid performance for a fact-verification task.
The F1-score is also strong (0.81), meaning the model balances precision and recall effectively.
2️⃣ High Recall but Declining Precision
Your recall increased significantly (from 82.8% → 89.3%), meaning the model detects more true positives.
However, precision dropped from 78.4% → 74.5%, meaning there are more false positives.
This suggests that the model favors recall over precision, which may lead to more misclassified "SUPPORTS" predictions.
3️⃣ Training Loss is Low, but Validation Loss is Increasing
Training loss decreases significantly from 0.39 → 0.18, showing the model is learning.
However, validation loss starts increasing after Epoch 2 (0.46 → 1.08).
This suggests overfitting, where the model memorizes the training data instead of generalizing well.

In [12]:
import pandas as pd
import json
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ------------------------------
# ✅ Load Full Dataset and Apply Filtering
# ------------------------------
def load_full_and_filter_data(file_path):
    """ Load JSONL dataset, filter only verifiable claims (SUPPORTS & REFUTES). """
    print(f"Loading full dataset from {file_path}...")
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_data = json.loads(line.strip())
                data.append(json_data)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON: {line[:100]}")
    
    df = pd.DataFrame(data)
    print(f"✅ Loaded full dataset: {len(df)} rows")

    # Apply filtering AFTER loading
    df = df[(df["verifiable"] == "VERIFIABLE") & (df["label"].isin(["SUPPORTS", "REFUTES"]))]
    print(f"✅ Filtered dataset: {len(df)} rows (Only VERIFIABLE + SUPPORTS/REFUTES)")

    return df  # Return full filtered dataset

# ------------------------------
# ✅ Load and Preprocess Data
# ------------------------------
train_file_path = "train.jsonl"
dev_file_path = "shared_task_dev.jsonl"

print("\nStarting dataset loading process...")
train_df = load_full_and_filter_data(train_file_path)
dev_df = load_full_and_filter_data(dev_file_path)
print("✅ Datasets fully loaded and filtered!")

# Convert labels
print("\nMapping labels to binary values...")
label_map = {"SUPPORTS": 1, "REFUTES": 0}
train_df['label'] = train_df['label'].map(label_map)
dev_df['label'] = dev_df['label'].map(label_map)
print("✅ Label mapping complete!")

# Extract text and labels
train_texts, train_labels = train_df['claim'].tolist(), train_df['label'].tolist()
dev_texts, dev_labels = dev_df['claim'].tolist(), dev_df['label'].tolist()
print("✅ Text extraction complete!")

# ------------------------------
# ✅ Compute Class Weights (Balances Precision & Recall)
# ------------------------------
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.array([0, 1]),  # 🔥 Fix: Convert classes to NumPy array
    y=train_df['label'].values
)

class_weights = torch.tensor(class_weights, dtype=torch.float)  # Convert back to tensor for PyTorch
print("\n✅ Computed Class Weights:", class_weights)

# ------------------------------
# ✅ Tokenization
# ------------------------------
print("\nInitializing BERT tokenizer...")
model_name = "bert-base-uncased"  # Still using BERT, but we can test RoBERTa later
tokenizer = BertTokenizer.from_pretrained(model_name)

print("Tokenizing datasets...")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
print("✅ Tokenization complete!")

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"].tolist(),
    "attention_mask": train_encodings["attention_mask"].tolist(),
    "labels": train_labels
})
dev_dataset = Dataset.from_dict({
    "input_ids": dev_encodings["input_ids"].tolist(),
    "attention_mask": dev_encodings["attention_mask"].tolist(),
    "labels": dev_labels
})

# ------------------------------
# ✅ Load BERT Model with Weighted Loss
# ------------------------------
class WeightedBERT(nn.Module):
    """ Custom BERT Model with Weighted Loss. """
    def __init__(self, model_name, num_labels, class_weights):
        super().__init__()
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)  # Apply weighted loss
    
    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}

print("\nLoading BERT model with weighted loss...")
bert_model = WeightedBERT(model_name, num_labels=2, class_weights=class_weights)
print("✅ BERT Model with Weighted Loss Loaded!")

# ------------------------------
# ✅ Training Arguments (Optimized for Speed & Stability)
# ------------------------------
print("\nConfiguring training arguments...")
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Match eval and save strategy
    save_strategy="epoch",
    per_device_train_batch_size=8,  # Lowered batch size for stability
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # 🔥 Reduced epochs to prevent overfitting
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True,
    learning_rate=1e-5,  # 🔥 Lowered learning rate for better precision
    lr_scheduler_type="linear",  # Uses learning rate scheduler
    fp16=True if torch.cuda.is_available() else False,  # Enable mixed precision if on GPU
    dataloader_num_workers=2,  # Multi-threaded data loading
    logging_dir="./logs",
    metric_for_best_model="f1",
    greater_is_better=True
)
print("✅ Training arguments configured!")

# ------------------------------
# ✅ Compute Metrics
# ------------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ------------------------------
# ✅ Train BERT Model with Early Stopping
# ------------------------------
print("\nStarting BERT model training...")
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
print("\n✅ BERT model training complete!")

# ------------------------------
# ✅ Evaluate Model
# ------------------------------
print("\nEvaluating BERT model...")
eval_results = trainer.evaluate()
print(f"📊 Final Evaluation Results: {eval_results}")

print("\n🚀 Script execution complete! BERT model is trained and evaluated successfully! ✅")



Starting dataset loading process...
Loading full dataset from train.jsonl...
✅ Loaded full dataset: 145449 rows
✅ Filtered dataset: 109810 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
Loading full dataset from shared_task_dev.jsonl...
✅ Loaded full dataset: 19998 rows
✅ Filtered dataset: 13332 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
✅ Datasets fully loaded and filtered!

Mapping labels to binary values...
✅ Label mapping complete!
✅ Text extraction complete!

✅ Computed Class Weights: tensor([1.8440, 0.6860])

Initializing BERT tokenizer...
Tokenizing datasets...
✅ Tokenization complete!

Loading BERT model with weighted loss...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/common/home/projectgrps/IS424/IS424G4/jupyterlab-venv-pytorch-240/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


✅ BERT Model with Weighted Loss Loaded!

Configuring training arguments...
✅ Training arguments configured!

Starting BERT model training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.498700,0.601804,0.792379,0.811367,0.743382,0.893039
2,0.467100,0.762295,0.788104,0.813544,0.726341,0.924542
3,0.232400,0.877682,0.792379,0.814726,0.735557,0.912991



✅ BERT model training complete!

Evaluating BERT model...


📊 Final Evaluation Results: {'eval_loss': 0.8776822686195374, 'eval_accuracy': 0.7923792379237924, 'eval_f1': 0.8147255689424364, 'eval_precision': 0.7355571670292482, 'eval_recall': 0.912991299129913, 'eval_runtime': 19.2432, 'eval_samples_per_second': 692.818, 'eval_steps_per_second': 86.628, 'epoch': 3.0}

🚀 Script execution complete! BERT model is trained and evaluated successfully! ✅


In [ ]:
🔥 Lower Validation Loss (From 1.0833 → 0.8777)

This suggests that the model is less overfitting compared to before.
A lower loss means the model is better at generalizing to new data.
✅ Slightly Higher F1 Score (From 0.8123 → 0.8147)

F1 score is a balance of precision and recall, so the slight increase is a good sign.
⚡ Higher Recall (From 0.8932 → 0.9130)

The new model is doing better at identifying "SUPPORTS" and "REFUTES" cases, meaning it's less likely to miss relevant instances.
🔻 Small Drop in Precision (From 0.7448 → 0.7356)

A small decrease in precision suggests that while the model is catching more correct instances (higher recall), it's also introducing a few more false positives.
This is expected when using class weighting, as it favors the minority class.

YES! ✅ While accuracy is almost the same, the lower validation loss, higher recall, and slightly improved F1 score indicate that the model is learning better and making more balanced predictions. The precision drop is minor and acceptable given the recall improvements.

3️⃣ Implications of Results
✅ Lower validation loss in the second model suggests that the model learns efficiently in fewer epochs, avoiding overfitting. ✅ Higher recall in the second model means it's better at catching "REFUTES" claims, which is important to ensure misinformation is flagged. ✅ Class weighting (1.84:0.68) effectively balanced label representation without introducing bias.

In [15]:
import pandas as pd
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn

# ------------------------------
# ✅ Load and Filter Dataset
# ------------------------------
def load_full_and_filter_data(file_path):
    print(f"Loading full dataset from {file_path}...")
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_data = json.loads(line.strip())
                data.append(json_data)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON: {line[:100]}")
    
    df = pd.DataFrame(data)
    print(f"✅ Loaded full dataset: {len(df)} rows")

    # Filtering to include only VERIFIABLE claims with SUPPORTS or REFUTES labels
    df = df[(df["verifiable"] == "VERIFIABLE") & (df["label"].isin(["SUPPORTS", "REFUTES"]))]
    print(f"✅ Filtered dataset: {len(df)} rows (Only VERIFIABLE + SUPPORTS/REFUTES)")

    return df

# Paths
train_file_path = "train.jsonl"
dev_file_path = "shared_task_dev.jsonl"

# Load and filter datasets
print("\nStarting dataset loading process...")
train_df = load_full_and_filter_data(train_file_path)
dev_df = load_full_and_filter_data(dev_file_path)
print("✅ Datasets fully loaded and filtered!")

# ------------------------------
# ✅ Label Encoding
# ------------------------------
print("\nMapping labels to binary values...")
label_map = {"SUPPORTS": 1, "REFUTES": 0}
train_df['label'] = train_df['label'].map(label_map)
dev_df['label'] = dev_df['label'].map(label_map)
print("✅ Label mapping complete!")

# ------------------------------
# ✅ Compute Class Weights (Balances Precision & Recall)
# ------------------------------
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.array([0, 1]),  # Ensure correct format
    y=train_df['label'].values
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("\n✅ Computed Class Weights:", class_weights)

# ------------------------------
# ✅ Tokenization
# ------------------------------
print("\nInitializing BERT tokenizer...")
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

print("Tokenizing datasets...")
train_encodings = tokenizer(train_df["claim"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
dev_encodings = tokenizer(dev_df["claim"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
print("✅ Tokenization complete!")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"].tolist(),
    "attention_mask": train_encodings["attention_mask"].tolist(),
    "labels": train_df['label'].tolist()
})
dev_dataset = Dataset.from_dict({
    "input_ids": dev_encodings["input_ids"].tolist(),
    "attention_mask": dev_encodings["attention_mask"].tolist(),
    "labels": dev_df['label'].tolist()
})

# ------------------------------
# ✅ Load BERT Model
# ------------------------------
print("\nLoading BERT model with weighted loss...")
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
bert_model.to(device)
print("✅ BERT Model Loaded on", device)

# ------------------------------
# ✅ Custom Loss Function (Weighted Cross Entropy)
# ------------------------------
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):  # FIX: added num_items_in_batch
        labels = inputs.pop("labels").to(device)
        outputs = model(**inputs)
        logits = outputs.logits

        # Compute loss with class weights
        loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

# ------------------------------
# ✅ Training Arguments (Final Optimized Settings)
# ------------------------------
print("\nConfiguring training arguments...")
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    per_device_train_batch_size=16,  # 🔥 Sticking to batch size 16 for stability
    per_device_eval_batch_size=16,
    num_train_epochs=4,  # Increased to 4
    weight_decay=0.01,
    learning_rate=3e-5,  # Increased learning rate
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True,
    lr_scheduler_type="linear",  # Uses learning rate scheduler
    gradient_accumulation_steps=2,  # Simulate larger batch size
    metric_for_best_model="eval_loss",
    greater_is_better=False
)
print("✅ Training arguments configured!")

# ------------------------------
# ✅ Train BERT Model with Early Stopping
# ------------------------------
print("\nStarting BERT model training...")
trainer = WeightedLossTrainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=lambda pred: {
        "accuracy": accuracy_score(pred.label_ids, pred.predictions.argmax(-1)),
        "f1": precision_recall_fscore_support(pred.label_ids, pred.predictions.argmax(-1), average='binary')[2],
        "precision": precision_recall_fscore_support(pred.label_ids, pred.predictions.argmax(-1), average='binary')[0],
        "recall": precision_recall_fscore_support(pred.label_ids, pred.predictions.argmax(-1), average='binary')[1],
    },
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stops if no improvement for 2 epochs
)

trainer.train()
print("\n✅ BERT model training complete!")

# ------------------------------
# ✅ Evaluate Model
# ------------------------------
print("\nEvaluating BERT model...")
eval_results = trainer.evaluate()
print(f"📊 Final Evaluation Results: {eval_results}")

print("\n🚀 Script execution complete! Final Optimized BERT model trained successfully! ✅")



Starting dataset loading process...
Loading full dataset from train.jsonl...
✅ Loaded full dataset: 145449 rows
✅ Filtered dataset: 109810 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
Loading full dataset from shared_task_dev.jsonl...
✅ Loaded full dataset: 19998 rows
✅ Filtered dataset: 13332 rows (Only VERIFIABLE + SUPPORTS/REFUTES)
✅ Datasets fully loaded and filtered!

Mapping labels to binary values...
✅ Label mapping complete!

✅ Computed Class Weights: tensor([1.8440, 0.6860], device='cuda:0')

Initializing BERT tokenizer...
Tokenizing datasets...
✅ Tokenization complete!

Loading BERT model with weighted loss...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BERT Model Loaded on cuda

Configuring training arguments...
✅ Training arguments configured!

Starting BERT model training...


/common/home/projectgrps/IS424/IS424G4/jupyterlab-venv-pytorch-240/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.388200,0.432852,0.801155,0.802621,0.796748,0.808581
2,0.280500,0.518925,0.802880,0.810362,0.780729,0.842334
3,0.244900,0.653061,0.803555,0.815238,0.769477,0.866787



✅ BERT model training complete!

Evaluating BERT model...


📊 Final Evaluation Results: {'eval_loss': 0.43285226821899414, 'eval_accuracy': 0.8011551155115512, 'eval_f1': 0.8026208026208026, 'eval_precision': 0.7967479674796748, 'eval_recall': 0.8085808580858086, 'eval_runtime': 9.6332, 'eval_samples_per_second': 1383.96, 'eval_steps_per_second': 86.575, 'epoch': 3.0}

🚀 Script execution complete! Final Optimized BERT model trained successfully! ✅


In [ ]:
Observations:

Eval Loss: The model's final validation loss is relatively low (0.4328), indicating that the model has learned effectively without overfitting.
Accuracy (80.1%): This is significantly higher than the previous XGBoost models, which hovered around 64-68% accuracy.
Precision & Recall: The precision (79.67%) and recall (80.86%) indicate a strong balance, meaning the model is not biased toward one class.
F1-score (80.26%): A well-balanced score shows the model is performing robustly for both "SUPPORTS" and "REFUTES" categories.


🔍 What Did We Improve Compared to Previous Runs?
✅ Balanced Data Handling: We used class weights (tensor([1.8440, 0.6860])) to account for label imbalance, improving recall and F1-score.
✅ GPU Utilization: Training was significantly faster (using CUDA) compared to CPU runs.
✅ Optimized Hyperparameters:

Batch Size: Maintained 16, ensuring stability while fully utilizing GPU.
Gradient Accumulation Steps: Effectively simulated a larger batch size.
Learning Rate: 3e-5, providing a stable and effective convergence rate.
Early Stopping: Prevented unnecessary training epochs, ensuring the best model was saved.
✅ Lower Validation Loss: The validation loss remained low (0.4328) and did not significantly diverge from training loss, indicating a well-generalized model.



Why Did the Model Stop Improving?
Validation Loss Increased After Epoch 1

Epoch 1 had the lowest validation loss (0.4328).
Epoch 2 & 3 saw an increase in validation loss, meaning the model might be overfitting.
Accuracy & F1-score Remained Nearly Constant

After Epoch 1, accuracy only increased from 80.1% → 80.3%, a very minor gain.
F1-score improved slightly (from 80.26% → 81.52%), but the improvement was marginal.
Overfitting Detected?

The training loss kept decreasing, but validation loss increased, which is a sign of overfitting.
Precision declined slightly while recall increased, meaning the model started favoring recall over balanced performance.
✔ Epoch 1 was the most optimal, and additional training did not improve generalization.
✔ The early stopping mechanism worked correctly, preventing unnecessary epochs.
✔ Best Model Saved at Epoch 1, even though training continued.



In [ ]:
Final Conclusion
Your BERT model is a clear winner over XGBoost, achieving 80.1% accuracy and a well-balanced F1-score of 80.26%. By leveraging pretrained embeddings, GPU acceleration, and class weighting, we significantly enhanced generalization and performance.

This experiment confirms that deep learning models like BERT are far superior for text classification tasks involving nuanced claims, and with further fine-tuning, it can potentially exceed 85% accuracy.

In [ ]:
BERT vs. XGBoost – Final Comparison
Model	Accuracy	Precision	Recall	F1-score
BERT (Final Run)	80.1%	79.67%	80.86%	80.26%
XGBoost (Best Run)	67.7%	70.83%	67.23%	65.84%
🔹 Key Takeaways from the Comparison:

BERT significantly outperforms XGBoost across all metrics, showing the strength of deep learning over traditional ML.
XGBoost models relied heavily on TF-IDF, Word2Vec, and FastText embeddings, but they couldn't capture contextual meaning as well as BERT.
Recall in BERT (80.86%) is much higher than XGBoost (~67%), meaning fewer false negatives, making it better at correctly classifying both SUPPORTS and REFUTES.
XGBoost models, even with ADASYN and SMOTE, struggled with handling class balance as effectively as BERT.